# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from pathlib import Path
import re
import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words

In [6]:
# load data from database
repo = Path.cwd()
path = repo / 'data' / 'DisasterResponse.db'

engine = create_engine(f'sqlite:///{path}')
df = pd.read_sql("select * from Message", con=engine)
df.shape

(25991, 38)

#check imbalance in target
See explore_data.ipynb

### 2. Write a tokenization function to process your text data

In [7]:
stop_words = stopwords.words("english")
vocab = set(words.words())
vocab = vocab - set(stop_words)
lemmatizer = WordNetLemmatizer()
words_set = {lemmatizer.lemmatize(word) for word in vocab}

In [8]:
def tokenize(text):
    """TO DO
    """
    
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(token).strip() for token in tokens if token in words_set]
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
    ("cv", CountVectorizer(tokenizer=tokenize)),
    ("transformer", TfidfTransformer()),
    ("rfc", MultiOutputClassifier(RandomForestClassifier(random_state=42, class_weight="balanced"), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X = df['message'].values
Y = df.iloc[:, :-3].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((20792,), (5199,), (20792, 35), (5199, 35))

In [12]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('cv',
                 CountVectorizer(tokenizer=<function tokenize at 0x1256afaf0>)),
                ('transformer', TfidfTransformer()),
                ('rfc',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        random_state=42),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_preds = pipeline.predict(X_test)

In [14]:
def get_metrics(y_test, y_pred):
    """TO DO
    """
    cr = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
    return cr

In [36]:
output_labels = list(df.iloc[:, :-3].columns)

def generate_report():
    report_dict = {}
    for idx, output in enumerate(output_labels):
        metrics_dict = get_metrics(Y_test[:,idx], Y_preds[:,idx])
        report_dict[output] = metrics_dict

#generate_report()

In [19]:
with open('rfc_model_results.json', 'w', encoding="utf-8") as fhand:
    json.dump(report_dict, fhand)

In [20]:
for k, v in report_dict.items():
    print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")

category: related 
 accuracy: 0.8221, precision: 0.8579, recall: 0.9209, f-score: 0.8883
category: request 
 accuracy: 0.8942, precision: 0.8150, recall: 0.5269, f-score: 0.6401
category: offer 
 accuracy: 0.9950, precision: 0.0000, recall: 0.0000, f-score: 0.0000
category: aid_related 
 accuracy: 0.7498, precision: 0.7111, recall: 0.6788, f-score: 0.6945
category: medical_help 
 accuracy: 0.9181, precision: 0.6098, recall: 0.1126, f-score: 0.1901
category: medical_products 
 accuracy: 0.9561, precision: 0.8214, recall: 0.0935, f-score: 0.1679
category: search_and_rescue 
 accuracy: 0.9738, precision: 0.6000, recall: 0.0435, f-score: 0.0811
category: security 
 accuracy: 0.9833, precision: 0.3333, recall: 0.0116, f-score: 0.0225
category: military 
 accuracy: 0.9686, precision: 0.6923, recall: 0.0536, f-score: 0.0994
category: water 
 accuracy: 0.9615, precision: 0.7606, recall: 0.5209, f-score: 0.6183
category: food 
 accuracy: 0.9427, precision: 0.8589, recall: 0.5953, f-score: 0.703

### 6. Improve your model
Use grid search to find better parameters. 

In [21]:
pipeline.get_params()

{'memory': None,
 'steps': [('cv',
   CountVectorizer(tokenizer=<function tokenize at 0x1256afaf0>)),
  ('transformer', TfidfTransformer()),
  ('rfc',
   MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                          random_state=42),
                         n_jobs=-1))],
 'verbose': False,
 'cv': CountVectorizer(tokenizer=<function tokenize at 0x1256afaf0>),
 'transformer': TfidfTransformer(),
 'rfc': MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                        random_state=42),
                       n_jobs=-1),
 'cv__analyzer': 'word',
 'cv__binary': False,
 'cv__decode_error': 'strict',
 'cv__dtype': numpy.int64,
 'cv__encoding': 'utf-8',
 'cv__input': 'content',
 'cv__lowercase': True,
 'cv__max_df': 1.0,
 'cv__max_features': None,
 'cv__min_df': 1,
 'cv__ngram_range': (1, 1),
 'cv__preprocessor': None,
 'cv__stop_words': None,
 'cv

In [22]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = param_grid = {
              "cv__max_df": (0.5, 1.0),  
              "rfc__estimator__min_samples_leaf": [5, 10],
#               "rfc__estimator__n_estimators": [100, 200],
#               "rfc__estimator__n_jobs": [-1],
              }

cv = GridSearchCV(pipeline, param_grid=param_grid, scoring="f1_micro")

In [33]:
cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('cv',
                                        CountVectorizer(tokenizer=<function tokenize at 0x1256afaf0>)),
                                       ('transformer', TfidfTransformer()),
                                       ('rfc',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                                               random_state=42),
                                                              n_jobs=-1))]),
             param_grid={'cv__max_df': (0.5, 1.0),
                         'rfc__estimator__min_samples_leaf': [5, 10]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
cv.best_params_

{'cv__max_df': 0.5, 'rfc__estimator__min_samples_leaf': 5}

In [35]:
Y_preds = cv.predict(X_test)

In [37]:
generate_report()

In [38]:
with open('rfc_gridsearchcv_results.json', 'w', encoding="utf-8") as fhand:
    json.dump(report_dict, fhand)

In [39]:
for k, v in report_dict.items():
    print(f"""category: {k} \n accuracy: {v['accuracy']:.4f}, precision: {v['1']['precision']:.4f}, recall: {v['1']['recall']:.4f}, f-score: {v['1']['f1-score']:.4f}""")

category: related 
 accuracy: 0.8221, precision: 0.8579, recall: 0.9209, f-score: 0.8883
category: request 
 accuracy: 0.8942, precision: 0.8150, recall: 0.5269, f-score: 0.6401
category: offer 
 accuracy: 0.9950, precision: 0.0000, recall: 0.0000, f-score: 0.0000
category: aid_related 
 accuracy: 0.7498, precision: 0.7111, recall: 0.6788, f-score: 0.6945
category: medical_help 
 accuracy: 0.9181, precision: 0.6098, recall: 0.1126, f-score: 0.1901
category: medical_products 
 accuracy: 0.9561, precision: 0.8214, recall: 0.0935, f-score: 0.1679
category: search_and_rescue 
 accuracy: 0.9738, precision: 0.6000, recall: 0.0435, f-score: 0.0811
category: security 
 accuracy: 0.9833, precision: 0.3333, recall: 0.0116, f-score: 0.0225
category: military 
 accuracy: 0.9686, precision: 0.6923, recall: 0.0536, f-score: 0.0994
category: water 
 accuracy: 0.9615, precision: 0.7606, recall: 0.5209, f-score: 0.6183
category: food 
 accuracy: 0.9427, precision: 0.8589, recall: 0.5953, f-score: 0.703

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [41]:
import pickle

pickle.dump(cv, open('model_rfc_gridsearchcv.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.